### Перед началом решения выполните следующую ячейку

In [72]:
!wget https://gist.github.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log

--2025-02-14 20:15:25--  https://gist.github.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log
Resolving gist.github.com (gist.github.com)... 140.82.121.4
Connecting to gist.github.com (gist.github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log [following]
--2025-02-14 20:15:27--  https://gist.githubusercontent.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 459418 (449K) [text/plain]
Saving to: 'auto_purchase.log'

     0K .......... .......... .......... .......... .......... 11% 3.40M 0s
    50K .......... .......... 

In [73]:
from datetime import datetime
from collections import defaultdict
import re


def parse_log_file(log_file_path):
    grouped = defaultdict(dict)
    with open(log_file_path, 'r') as file:
        for line in file:    
            splited = line.split('|')
            key = datetime.strptime(splited[1].strip(), "%Y-%m-%d %H:%M:%S,%f").date()
            user_id = re.search('( id: )(\d*)', splited[4])
            if user_id is not None and 'Обновляем подписку' in splited[4]:
                grouped[key].setdefault('sub', list()).append(int(user_id.group(2)))
            elif user_id is not None and 'У пользователя' in splited[4]:
                if splited[0].startswith('INFO'):
                    grouped[key].setdefault('exp', list()).append(int(user_id.group(2)))
                else:
                    grouped[key].setdefault('err', list()).append(int(user_id.group(2)))
            elif key not in grouped:
                grouped[key] = {}
    return grouped

### Задача 1

In [74]:
def count_success_and_failure(log_file_path):
    parsed_log = parse_log_file(log_file_path)

    total_sub = sum(len(v.get('sub', [])) for v in parsed_log.values())
    total_err = sum(len(v.get('err', [])) for v in parsed_log.values())

    return (total_sub - total_err, total_err)

**Проверка**

In [75]:
res = count_success_and_failure('auto_purchase.log')

try:
    assert res == (1034, 186)
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### Задача 2

![image info](chapter14case1task2.png)

**Решением с добавлением записи об обновлении подписки для пользователя**

In [76]:
def auto_renewal_sub(log_file_path):
    parsed_log = parse_log_file(log_file_path)

    strange_date = datetime.strptime('2022-12-31', "%Y-%m-%d").date()
    parsed_log[strange_date]['sub'].append(666)
    
    values = [len({*v.get('sub', [])}) for v in parsed_log.values()]
    means = []
    medians = []
    for i in range(len(values)):
        current_window = values[:i+1]
        
        # среднее
        mean = sum(current_window) / len(current_window)
        means.append(round(mean, 2))
        
        # медианa
        sorted_window = sorted(current_window)
        if len(current_window) % 2 == 0:
            median = (sorted_window[len(current_window)//2 - 1] + 
                    sorted_window[len(current_window)//2]) / 2
        else:
            median = sorted_window[len(current_window)//2]
        medians.append(int(median))
        
    with open('auto_renewal_sub.txt', 'w') as file:
        file.write(f"Среднее: {means}\n")
        file.write(f"Медиана: {medians}\n")
    

auto_renewal_sub('auto_purchase.log')

**Решение основывающееся на разбоер сообщения формата: "*(\d{4})-(\d{2})-(\d{2}).\*: (\d\*)$*"**

In [79]:
from datetime import datetime
from collections import defaultdict
import re


def auto_renewal_sub(log_file_path):
    grouped = defaultdict(int)
    with open(log_file_path, 'r') as file:
        for line in file:
            splited = line.split('|')
            key = datetime.strptime(splited[1].strip(), "%Y-%m-%d %H:%M:%S,%f").date()
            user_qty = re.search(r'(\d{4})-(\d{2})-(\d{2}).*: (\d*)$', splited[4])
            if user_qty is not None:
                qty_value = re.search(r'(\d*)$', user_qty.group(0))
                grouped[key] = max(grouped[key], int(qty_value.group(0)))            

    values = [*grouped.values()]
    means = []
    medians = []
    for i in range(len(values)):
        current_window = values[:i+1]
        
        # среднее
        mean = sum(current_window) / len(current_window)
        means.append(round(mean, 2))
        
        # медианa
        sorted_window = sorted(current_window)
        if len(current_window) % 2 == 0:
            median = (sorted_window[len(current_window)//2 - 1] + 
                    sorted_window[len(current_window)//2]) / 2
        else:
            median = sorted_window[len(current_window)//2]
        medians.append(int(median))
        
    with open('auto_renewal_sub.txt', 'w') as file:
        file.write(f"Среднее: {means}\n")
        file.write(f"Медиана: {medians}\n")

auto_renewal_sub('auto_purchase.log')

**Проверка**

In [77]:
!wget https://gist.github.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt

--2025-02-14 20:15:48--  https://gist.github.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt
Resolving gist.github.com (gist.github.com)... 140.82.121.4
Connecting to gist.github.com (gist.github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt [following]
--2025-02-14 20:15:48--  https://gist.githubusercontent.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2431 (2.4K) [text/plain]
Saving to: 'cor_auto_renewal.txt'

     0K ..                                                    100% 11.1M=0s

2025-02-14 20:15:49

In [80]:
import pandas as pd

user_answer = pd.read_csv('auto_renewal_sub.txt')
correct_answer = pd.read_csv('cor_auto_renewal.txt')

try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!


### Задача 3

In [81]:
from collections import defaultdict

def sub_renewal_by_day(log_file_path):
    parsed_log = parse_log_file(log_file_path)
    days_of_week = ['Понедельник', 'Вторник', 'Среда', 'Четверг', 'Пятница', 'Суббота', 'Воскресенье']
    week_dict = defaultdict(int)
    for k, v in parsed_log.items():
        key = k.weekday()
        value = week_dict.get(key, [0, 0])
        value[0] += len(v.get('sub', []))
        value[1] += len(v.get('err', []))
        week_dict[key] = value

    with open('weekdays.txt', 'w') as file:
        file.write(f"Количество обновлений подписки по дням недели:\n")    
        file.write('\n'.join([f'{d}: {week_dict[i][0] - week_dict[i][1]}' for i, d in enumerate(days_of_week)]))
        
sub_renewal_by_day('auto_purchase.log')

**Проверка**

In [82]:
!wget https://gist.github.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt

--2025-02-14 20:16:11--  https://gist.github.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt
Resolving gist.github.com (gist.github.com)... 140.82.121.4
Connecting to gist.github.com (gist.github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt [following]
--2025-02-14 20:16:11--  https://gist.githubusercontent.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238 [text/plain]
Saving to: 'cor_weekdays.txt'

     0K                                                       100% 22.7M=0s

2025-02-14 20:16:11 (22.7 MB/s) - 'cor_week

In [83]:
import pandas as pd

user_answer = pd.read_csv('weekdays.txt')
correct_answer = pd.read_csv('cor_weekdays.txt')

try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!
